# Bayesian Bandit A/A-Testing

In [1]:
# !which python
# !pip install nbformat
# !pip install kaleido
# !makedir images
# !makedir video

In [2]:
from typing import Dict, List, Any, Union

import numpy as np
import pandas as pd
import math

from tqdm import tqdm

from scipy import stats
from scipy.stats import beta, gamma

# import util functions
from bayesian_bandit_test import Environment, Agent, Bandit
from bayesian_test import Bayesian_AB_Test

from graph import visualisation # conda install -n python3 -c conda-forge colorlover
from graph import Video
import plotly
import plotly.graph_objects as go
# Init visualisation tool
plot = visualisation(renderer="vscode") # vscode | iframe for browsers

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 10000)

tqdm.pandas()

# Parameters

In [3]:
# A/A-Test
BANDIT_PARAMS = {'A1': {'period':0, 'ctr':0.1, 'cpm':1},
                 'A2': {'period':0, 'ctr':0.1, 'cpm':1}, 
                 'B1': {'period':0, 'ctr':0.3, 'cpm': 2},
                 'B2': {'period':0, 'ctr':0.3, 'cpm': 2}}

# Plotting
WIDTH_SAVE, HEIGHT_SAVE = 1200, 400

In [4]:
# config = {'optimise_for': 'ctr',
#           'n_periods': 300,
#           'max_impr_before_update_param': 100,
#           'recency_param': 0.6, # decay parameter`per day`
#           'n_periods_per_day': 24, # number of periods per day
#           'video': 'video/bandit_aa_ctr_slow.mp4'
#          }

config = {'optimise_for': 'ctr',
          'n_periods': 300,
          'max_impr_before_update_param': 5000,
          'recency_param': 0.95, # decay parameter`per day`
          'n_periods_per_day': 1, # number of periods per day
          'video': 'video/bandit_aa_ctr_fast.mp4'
         }

bandit = Bandit(bandit_params=BANDIT_PARAMS, n_periods=config['n_periods']+1, config=config)
bandit.run()
bandit.agent.df_log['A1'].tail(5)
bandit.agent.df_log['A2'].tail(5)
bandit.agent.df_log['B1'].tail(5)
bandit.agent.df_log['B2'].tail(5)
bandit.df_metrics.tail()

,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
297,297,0,52.892238,0,7.528096,0.000000,NaN,0.142329,0.147805,9,46,2.112687,0.132836,1.112687
298,298,25,50.297626,2,7.201691,0.025000,NaN,0.143182,0.153721,8,44,2.107053,0.138856,1.107053
299,299,23,71.582745,3,8.791606,0.052848,NaN,0.122817,0.128014,10,64,2.125450,0.113745,1.125450
300,300,0,89.903608,0,11.252026,0.000000,NaN,0.125157,0.103926,12,80,2.169383,0.088873,1.169383
301,301,0,85.458427,0,10.739425,0.000000,NaN,0.125668,0.108098,12,76,2.160914,0.093115,1.160914


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
297,297,0,64.075482,0,7.888420,0.000000,NaN,0.123111,0.140787,9,57,2.110591,0.126768,1.110591
298,298,4,60.921708,1,7.543999,0.004000,NaN,0.123831,0.146482,9,54,2.105061,0.132556,1.105061
299,299,3,61.725622,0,8.166799,0.005228,NaN,0.132308,0.135134,9,55,2.103608,0.122447,1.103608
300,300,1,61.539341,1,7.808459,0.001000,NaN,0.126886,0.141308,9,55,2.103394,0.128066,1.103394
301,301,1,59.462374,0,8.418036,0.001000,NaN,0.141569,0.130574,9,52,2.099175,0.118793,1.099175


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
297,297,141,16749.202106,55,5044.675869,0.512980,NaN,0.301189,0.006285,5046,11706,32.707424,0.000198,31.707424
298,298,2090,16045.742000,614,4844.742075,2.090000,NaN,0.301933,0.006328,4846,11202,31.659385,0.000206,30.659385
299,299,1506,17229.004900,458,5185.854971,4.549521,NaN,0.300996,0.006009,5187,12044,32.161915,0.000193,31.161915
300,300,1562,17798.304655,443,5361.712223,4.934691,NaN,0.301248,0.006337,5363,12438,34.975864,0.000187,33.975864
301,301,244,18392.339423,49,5514.526612,0.713792,NaN,0.299827,0.006712,5516,12879,38.015028,0.000181,37.015028


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
297,297,168,23685.461174,57,7153.802234,0.168000,NaN,0.302033,0.008314,7155,16533,60.475172,0.000140,59.475172
298,298,2712,22660.838115,813,6850.312122,12.561925,NaN,0.302297,0.008279,6851,15812,57.711014,0.000146,56.711014
299,299,1697,24104.246209,518,7280.196516,1.697000,NaN,0.302030,0.009046,7281,16825,66.859291,0.000137,65.859291
300,300,2228,24511.233899,647,7408.336690,3.110119,NaN,0.302243,0.008670,7409,17104,65.228477,0.000135,64.228477
301,301,396,25402.322204,116,7652.619856,0.396000,NaN,0.301257,0.008366,7654,17751,65.021665,0.000131,64.021665


,period,n_impr,regret,P_ab_ctr,P_ab_cpc,loss_ctr,loss_cpc,p_overlap_ctr,p_overlap_cpc,n_impr_acc,regret_acc,regret_avg
296,297,309,168,"[0.0067, 0.0006, 0.4182, 0.5745]","[0.0, 0.0, 0.0019, 0.9981]","{('A1', 'A2'): (0.014045522798786313, 0.041045...","{('A1', 'A2'): (0.09990476375363369, 0.1009911...","{'ks': {('A1', 'A2'): 0.968, ('A1', 'B1'): 0.0...",{'ks': {}},707638,409251,0.578334
297,298,4831,2741,"[0.0063, 0.0007, 0.4664, 0.5266]","[0.0, 0.0, 0.003, 0.997]","{('A1', 'A2'): (0.02076275962725056, 0.0316418...","{('A1', 'A2'): (0.09985065973032325, 0.1009615...","{'ks': {('A1', 'A2'): 0.983, ('A1', 'B1'): 0.0...",{'ks': {}},712469,411992,0.578260
298,299,3229,1723,"[0.0001, 0.0012, 0.4115, 0.5872]","[0.0, 0.0, 0.0003, 0.9997]","{('A1', 'A2'): (0.025688324123937435, 0.020253...","{('A1', 'A2'): (0.09952065758193292, 0.1011283...","{'ks': {('A1', 'A2'): 0.965, ('A1', 'B1'): 0.0...",{'ks': {}},715698,413715,0.578058
299,300,3791,2229,"[0.0, 0.0006, 0.4149, 0.5845]","[0.0, 0.0, 0.0008, 0.9992]","{('A1', 'A2'): (0.027061853129577682, 0.016973...","{('A1', 'A2'): (0.09843069633249495, 0.1016151...","{'ks': {('A1', 'A2'): 0.944, ('A1', 'B1'): 0.0...",{'ks': {}},719489,415944,0.578110
300,301,641,397,"[0.0002, 0.0021, 0.3713, 0.6264]","[0.0, 0.0, 0.0035, 0.9965]","{('A1', 'A2'): (0.028554888551413728, 0.017489...","{('A1', 'A2'): (0.09784561694754478, 0.1022754...","{'ks': {('A1', 'A2'): 0.945, ('A1', 'B1'): 0.0...",{'ks': {}},720130,416341,0.578147


# Plotting

In [5]:
def extract_period(df: pd.DataFrame, period: int) -> pd.DataFrame:
    """ Extract data for given period
    """
    return {variant: df[variant][df[variant].period==period] for variant in df.keys() if sum(df[variant].period==period)>0}

In [6]:
# Impressions / Clicks over time
df = bandit.agent.df_log.copy()

p_data = []
for i, variant in enumerate(bandit.agent.variants):
    p_data += [ plot.plot(x=df[variant].period, y=df[variant].n_impr_w_sum, color=i, opacity=0.4, name=f'impr. {variant}', showlegend=True),
                plot.plot(x=df[variant].period, y=df[variant].n_clicks_w_sum, color=i, opacity=0.7, name=f'clicks {variant}', showlegend=True)]
layout = plot.layout(title=f'Observations - impr. & clicks', x_label='time', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
# layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
# go.Figure(data=p_data, layout=layout).write_image('images/impr_clicks.png')

In [7]:
PERIOD = 137

df_T = extract_period(df=bandit.agent.df_log, period=PERIOD)

# Click-Through-Rate - Beta distribution
for variant in df_T:
    print(variant)
    df_T[variant]

x = np.linspace(0, 1, 1000)
p_data = [plot.plot(x=x, y=beta.pdf(x, df_T[variant].alpha, df_T[variant].beta), color=i, opacity=0.7, name=variant, showlegend=True) for i, variant in enumerate(df_T)]
layout = plot.layout(title=f'Beta distributions at T:{PERIOD}', x_label='Click-Through-Rate', y_label='p', theme='dark', width=1200, height=400)
layout['xaxis']['range'] = [0, 0.5]
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image('images/bandit_beta_aa.png')

# Cost-per-Click - gamma distribution
x = np.linspace(0, 20, 1000)
p_data = [plot.plot(x=x, y=gamma.pdf(x, a=df_T[variant].a, scale=df_T[variant].scale), color=i, opacity=0.7, name=variant, showlegend=True) for i, variant in enumerate(df_T)]
layout = plot.layout(title=f'Gamma distributions at T:{PERIOD}', x_label='Cost-per-Click', y_label='p', theme='dark', width=1200, height=400)
layout['xaxis']['range'] = [0, 20]
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image('images/bandit_gamma_aa.png')

A1


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
137,137,0,52.323072,0,5.253669,0.0,NaN,0.100408,0.210369,6,48,2.105207,0.190343,1.105207


A2


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
137,137,0,61.855423,0,7.831435,0.0,NaN,0.126609,0.144616,9,55,2.132554,0.127691,1.132554


B1


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
137,137,1309,30319.348897,407,9099.29727,1.309,NaN,0.300115,0.006162,9100,21221,57.066874,0.00011,56.066874


B2


,period,n_impr,n_impr_w_sum,n_clicks,n_clicks_w_sum,cost,cost_sum,ctr,cpc,alpha,beta,a,scale,cost_w_sum
137,137,251,13453.773246,80,3968.544671,1.245249,NaN,0.294976,0.008315,3970,9486,33.999209,0.000252,32.999209


### A/A-test - Kolmogorov-Smirnof

In [8]:
df = pd.DataFrame(bandit.df_metrics.p_overlap_ctr.to_list())
df_ks = pd.DataFrame(df.ks.to_list()) # KS-test
df_ws = pd.DataFrame(df.ws.to_list()) # WS-test
df = df_ks
# df_ks.head(10)
# df.columns

# A/A-test - KS
p_data = [
          plot.plot(x=list(range(len(df))), y=df[('A1','A2')], color=0, opacity=0.5, name='A1 - A2', showlegend=True),
          plot.plot(x=list(range(len(df))), y=df[('B1','B2')], color=1, opacity=0.7, name='B1 - B2', showlegend=True),
        #   plot.plot(x=list(range(len(df))), y=df[('A1','B1')], color=1, opacity=0.5, name='A1 - B1', showlegend=True),
        #   plot.plot(x=list(range(len(df))), y=df[('A2','B2')], color=1, opacity=0.5, name='A2 - B2', showlegend=True),        
          ]
layout = plot.layout(title=f'KS - AA-test', x_label='Cost-per-Click', y_label='p', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
layout['width'], layout['height'] = WIDTH_SAVE, HEIGHT_SAVE
go.Figure(data=p_data, layout=layout).write_image('images/bandit_aa_ctr_ks.png')

In [9]:
# Regret over time
p_data = [ plot.plot(x=bandit.df_metrics.period, y=bandit.df_metrics.regret, color=0, opacity=0.9, name=f'regret', showlegend=True)]
layout = plot.layout(title=f'Regret', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

# Regret - CDF
hist, bins = np.histogram(bandit.df_metrics.regret, bins=100)
p_data = [ plot.plot(x=bins, y=hist, color=0, opacity=0.6, name=f'regret', showlegend=True)]
layout = plot.layout(title=f'Regret - Distribution', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()
p_data = [ plot.plot(x=bins, y=np.cumsum(hist)/sum(hist), color=0, opacity=0.6, name=f'regret', showlegend=True)]
layout = plot.layout(title=f'Regret - CDF', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

#### CTR

In [10]:
# P(A>B)
# map to dataframe, where each row is a period and each column is a variant
df_p_ab = pd.DataFrame(bandit.df_metrics.P_ab_ctr.to_list(), columns=bandit.agent.variants)

p_data = [ plot.plot(x=bandit.df_metrics.period, y=df_p_ab[variant], color=i, opacity=0.7, name=f'P - {variant}', showlegend=True) for i, variant in enumerate(df_p_ab.columns)]
layout = plot.layout(title=f'p_ab', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

# Loss
# map loss_ctr, where each row is a period and each column is a variant
df_loss = pd.DataFrame(bandit.df_metrics.loss_ctr.to_list())
df_loss = df_loss.applymap(lambda x: (0, 0) if pd.isna(x) else x)

for i, variant in enumerate(df_loss.columns):
    tmp1 = df_loss[variant].apply(lambda x: x[0])
    tmp2 = df_loss[variant].apply(lambda x: x[1])
    p_data = [ plot.plot(x=bandit.df_metrics.period, y=tmp1, color=0, opacity=0.7, name=f'P - {variant} - A', showlegend=True),
               plot.plot(x=bandit.df_metrics.period, y=tmp2, color=1, opacity=0.7, name=f'P - {variant} - B', showlegend=True) ]
    layout = plot.layout(title=f'loss', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
    fig = go.Figure(data=p_data, layout=layout).show()

#### CpC

In [11]:
# P(A>B)
# map to dataframe, where each row is a period and each column is a variant
df_p_ab = pd.DataFrame(bandit.df_metrics.P_ab_cpc.to_list(), columns=bandit.agent.variants)

p_data = [ plot.plot(x=bandit.df_metrics.period, y=df_p_ab[variant], color=i, opacity=0.7, name=f'P - {variant}', showlegend=True) for i, variant in enumerate(bandit.agent.variants)]
layout = plot.layout(title=f'p_ab', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
fig = go.Figure(data=p_data, layout=layout).show()

# Loss
# map loss_ctr, where each row is a period and each column is a variant
df_loss = pd.DataFrame(bandit.df_metrics.loss_cpc.to_list())
df_loss = df_loss.applymap(lambda x: (0, 0) if pd.isna(x) else x)

for i, variant in enumerate(df_loss.columns):
    tmp1 = df_loss[variant].apply(lambda x: x[0])
    tmp2 = df_loss[variant].apply(lambda x: x[1])
    p_data = [ plot.plot(x=bandit.df_metrics.period, y=tmp1, color=0, opacity=0.7, name=f'P - {variant} - A', showlegend=True),
               plot.plot(x=bandit.df_metrics.period, y=tmp2, color=1, opacity=0.7, name=f'P - {variant} - B', showlegend=True) ]
    layout = plot.layout(title=f'loss', x_label='periods', y_label='#', theme='dark', width=1200, height=400)
    fig = go.Figure(data=p_data, layout=layout).show()

<hr>

### Video

In [12]:
# Bandit - AA-etst - CTR
N_STEPS = bandit.df_metrics.shape[0]-1

colormap = ['#ff0000', '#ff0000', '#ff00ff', '#ff00ff']
video = Video(xlabel='CTR', x_lim=0.5, y_lim=200, n_versions=4, colormap=colormap, txt_pos=0.5)
with video.writer.saving(video.fig, config['video'], 200):
    x = np.linspace(0, 1, 50000)
    for period in tqdm(range(N_STEPS+1)):
        df_T = extract_period(df=bandit.agent.df_log, period=period)
        txt = 'Period: {}\n\nClicks  |  Impressions\n'.format(period)
        for i, variant in enumerate(df_T):
            video.plts[i].set_data(x, beta.pdf(x, df_T[variant].alpha.values[0], df_T[variant].beta.values[0]))
        
            txt += '{}: {: >8.1f}  |  {: >8.1f}\n'.format(variant,
                                                    df_T[variant].n_clicks_w_sum.values[0],
                                                    df_T[variant].n_impr_w_sum.values[0],
                                                    )

        txt += '\nks - AA: {: >8.1f}\n'.format(100*bandit.df_metrics.p_overlap_ctr[period]['ks'][('A1','A2')])
        txt += 'ks - BB: {: >8.1f} '.format(100*bandit.df_metrics.p_overlap_ctr[period]['ks'][('B1','B2')])

        video.txt_time.set_text(txt)
        video.writer.grab_frame(facecolor=video.fig.get_facecolor(), edgecolor='none')
print('Completed movie: {}'.format(config['video']))

100%|██████████| 301/301 [00:28<00:00, 10.63it/s]


Completed movie: video/bandit_aa_ctr_fast.mp4
